# Image Preporcessing

There are a few steps for image preprocessing so we can use them for training.
1. Image Extraction: Video recordings of a person will be extracted frame by frame and the frames will be saved as images for subsequent process
2. Face Alignment: The images extrated will go through face detection model to extract the faces with a predefined marginal space
3. Face Encoding: The headshots will be fed into the FaceNet model in order to get the embedding for the face. **Note: this preprocessing might be integrated into training if we are doing fine-tuning where we are adjusting the top layer of the FaceNet model.**

In [ ]:
#impoart packages used for visulization in this notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## 1. Image Extraction

In [ ]:
# Import packages
import os
from os.path import isdir
from os.path import join

import cv2

**Set the arguments. When running the code as script, one will have to use argparse or similar package to feed to arguments into the script**

In [ ]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"data/videos",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted images are saved
    "output_dir":"data/images",
    "split_types":["train", "val", "test"],
    # Define how many frames to skip before extract each frame 
    "skip":100
}

**Extract frames and save as images**

In [ ]:
for split in args["split_types"]:
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    # print(subdirs)
    for subdir in subdirs:
        output_subdir = join(args["output_dir"], split, subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)
        for video in os.listdir(join(input_dir,subdir)):
            video_path = join(input_dir,subdir, video)
            vs = cv2.VideoCapture(video_path)
            read = 0
            saved = 0
            # loop over frames from the video file stream
            while True:
                # grab the frame from the file
                (grabbed, frame) = vs.read()
                # if the frame was not grabbed, then we have reached the end
                # of the stream
                if not grabbed:
                    break
                # increment the total number of frames read thus far
                read += 1
                # check to see if we should process this frame
                if read % args["skip"] != 0:
                    continue
                output_img_path = join(output_subdir,"{}.png".format(saved))
                cv2.imwrite(output_img_path, frame)
                saved += 1
                print("[INFO] saved {} to disk".format(output_img_path))

## Face Alignment

Once we have a bunch of images, we can align the faces in those images and extract them, and again save them as images.

In [ ]:
# Import packages
import os
from os.path import isdir
from os.path import join
import importlib

import numpy as np
import cv2

from src import detectors
# from common_resources.detectors import FaceDetector


importlib.reload(detectors)
FaceDetector = detectors.FaceDetector

We have a few options for face detections:
1. The detector in HW7
2. Open CV detector

Build a wrapper so we can use them in the same way

In [ ]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"data/images/",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted faces are saved
    "output_dir":"data/faces/",
    # Define the margin (extra space) for face alignment
    "margin":10,
    "split_types":["train", "val", "test"],
    # Define the face detector used and paramters
    "detector": ("DLIB",)    
}

In [ ]:
dlib_face_detector = FaceDetector(args["detector"])

In [ ]:
# Initialize the detector of choice
face_detector = FaceDetector(args["detector"])
margin = args["margin"]

for split in args["split_types"]:
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    
    for subdir in subdirs:
        output_subdir = join(args["output_dir"], split, subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)
        saved = 0
        for file in os.listdir(join(input_dir, subdir)):
            image_path = join(input_dir, subdir, file)
            image = cv2.imread(image_path)

            faces = face_detector.detect_faces(image)
            for (x,y,w,h) in faces:
                x_start = max(0,x-margin//2)
                y_start = max(0,y-margin//2)
                x_end = min(image.shape[1], x+w+margin//2)
                y_end = min(image.shape[0], y+h+margin//2)
                cropped = image[y_start:y_end, x_start:x_end]
                # resize the image to 160x160 for FaceNet
                aligned = cv2.resize(cropped, (160, 160))
                output_img_path = join(output_subdir,"face_{}.png".format(saved))
                cv2.imwrite(output_img_path, aligned)
                saved += 1
                print("[INFO] saved {} to disk".format(output_img_path))

## Face Encoding

Now that we have the faces extracted and aligned, we could encode the faces with the FaceNet model for training the SVM. Note: this step is not necessary for fine-tuning on a neural classifier

In [ ]:
# Import packages
import os
from os.path import isdir
from os.path import join
import importlib

import numpy as np
import cv2

from src import face_encoders

importlib.reload(face_encoders)
FaceEncoder = face_encoders.FaceEncoder

In [ ]:
# change the model file to the tuned one if tuned FaceNet
encoder_model = FaceEncoder("facenet_keras", "src/encoders/facenet_keras.h5")

In [ ]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"data/faces/",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted faces are saved
    "output_dir":"data/embeddings/procees_1",
    "split_types":["train", "val", "test"],
}

In [ ]:
if not os.path.exists(args["output_dir"]):
    os.makedirs(args["output_dir"])

for split in args["split_types"]:
    print(f"[INFO] Encoding faces in {split} set...")
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    
    faces = list()
    embeddings = list()
    labels = list()
    for label in subdirs:
        files = os.listdir(join(input_dir, label))
        print(f'    [INFO] Encoding {len(files)} "{label}" faces...')
        for file in files:
            face_path = join(input_dir, label, file)
            face = cv2.imread(face_path)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face_array = np.asarray(face)
            embedding = encoder_model.get_embedding(face_array)
            faces.append(face_array)
            embeddings.append(embedding)
            labels.append(label)
    faces = np.asarray(faces)
    embeddings = np.asarray(embeddings)
    labels = np.asarray(labels)
    output_face_file = join(args["output_dir"], f"{split}_faces.npz")
    output_emb_file = join(args["output_dir"], f"{split}_embeddings.npz")
    np.savez_compressed(output_face_file, faces, labels)
    np.savez_compressed(output_emb_file, embeddings, labels)
    print(f"[INFO] Saved {split} set as {output_face_file}, shape: {str(embeddings.shape)}, {str(labels.shape)}")